In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from math import prod

In [2]:
from zipfile import ZipFile
with ZipFile('newsgroups.zip', "r") as z:
    z.extractall()

In [3]:
with open("newsgroups.txt", "r") as foo:
     news = foo.read().split("Newsgroup:")

In [4]:
print(len(news))

7379


In [5]:
news = news[1:]

In [6]:
newsgroups = []
From = []
subjects = []
Body = []

for i in range(len(news)):
    post = news[i].split("\n")
    group = post[0].strip()
    if "From" in post[1] and "Subject" in post[2]:
        source = post[1].split("From: ")[1]
        subject = post[2].split("Subject: ")[1].strip()
    body = "".join(post[3:]).replace(">", "").strip()
    
    newsgroups.append(group)
    From.append(source)
    subjects.append(subject)
    Body.append(body)

In [7]:
df = pd.DataFrame()

In [8]:
df["newsgroup"] = newsgroups
df["from"] = From
df["subject"] = subjects
df["body"] = Body

In [9]:
df

,newsgroup,from,subject,body
0,rec.autos,gwm@spl1.spl.loral.com (Gary W. Mahan),Re: Are BMW's worth the price?,sure sounds like they got a ringer. the 325is...
1,sci.med,davec@ecst.csuchico.edu (Dave Childs),Dental Fillings question,I have been hearing bad thing about amalgam de...
2,alt.atheism,"""Robert Knowles"" <p00261@psilink.com>",Re: Islamic marriage?,"DATE: Tue, 6 Apr 1993 00:11:49 GMTFROM: F...."
3,rec.sport.baseball,sepinwal@mail.sas.upenn.edu (Alan Sepinwall),Re: WFAN,In article <1993Apr16.174843.28111@cabell.vcu....
4,talk.religion.misc,rwd4f@poe.acc.Virginia.EDU (Rob Dobson),Re: A Message for you Mr. President: How do yo...,In article <visser.735284180@convex.convex.com...
...,...,...,...,...
7373,sci.med,ron.roth@rose.com (ron roth),Selective Placebo,L( levin@bbn.com (Joel B Levin) writes:L( Jo...
7374,rec.autos,RZAA80@email.sps.mot.com (Jim Chott),Re: Toyota Land Cruiser worth it?,"In article <1r3sbbINN8e0@hp-col.col.hp.com, tv..."
7375,rec.sport.hockey,"""James J. Murawski"" <jjm+@andrew.cmu.edu>",This Year's vs. Next Year's Playoffs,"Well, since someone probably wanted to know, h..."
7376,rec.sport.hockey,au021@cleveland.Freenet.Edu (Roland Behunin),Does anybody have the schedule for games Sunda...,"Hello Hockey fans.Bonjour tout le monde!Well, ..."


In [10]:
df["newsgroup"].unique()

array(['rec.autos', 'sci.med', 'alt.atheism', 'rec.sport.baseball',
       'talk.religion.misc', 'rec.sport.hockey', 'sci.electronics',
       'rec.motorcycles'], dtype=object)

In [11]:
train_frac = 0.5
train_df = df.sample(frac=train_frac, random_state=0).copy()
train_df

,newsgroup,from,subject,body
1940,sci.med,balick@nynexst.com (Daphne Balick),Re: Altitude adjustment,In article <4159@mdavcr.mda.ca vida@mdavcr.mda...
4532,rec.sport.hockey,Martin D. Hill <32GFKKH@CMUVM.CSV.CMICH.EDU>,Re: NHL team in Milwaukee,"Well put, Jason. I am not from Wisconsin, but..."
3774,rec.motorcycles,edw@boi.hp.com (Edward Walsh),"Guzzi 850-T tires, plus fairing question",I have a question about tires for my Guzzi. It...
4154,rec.motorcycles,rwf2@ns1.cc.lehigh.edu (ROBERT WILLIAM FUSI),Re: Recommendation for a front tire.,"In article <C5HrxJ.ACE@unix.amherst.edu, nrmen..."
5204,rec.sport.baseball,klassen@sol.UVic.CA (Melvin Klassen),Re: Is Western insane??,In article <93108.1841463321628@QUCDN.QueensU....
...,...,...,...,...
2491,rec.autos,Tom Wetzel <twetzel@ucs.indiana.edu>,Re: Old Corvettes / Low insurance?,In article <rjnC5sxMG.JqH@netcom.com Richard N...
2859,alt.atheism,keith@cco.caltech.edu (Keith Allan Schneider),"Re: ""Cruel"" (was Re: <Political Atheists?)",kmr4@po.CWRU.edu (Keith M. Ryan) writes:They s...
4595,alt.atheism,kmr4@po.CWRU.edu (Keith M. Ryan),Re: Nicknames,In article <16BB6B6FE.I3150101@dbstu1.rz.tu-bs...
4346,alt.atheism,"dps@nasa.kodak.com (Dan Schaertel,,,)",Re: Christian Morality is,"In article 11853@vice.ICO.TEK.COM, bobbe@vice...."


In [12]:
groups = train_df.groupby(by="newsgroup")

In [13]:
def wc(text):
    words = re.findall("[a-z']+", text.lower())
    return Counter(words)

In [14]:
train_df["word_counts"] = train_df["body"].map(wc)
train_df

,newsgroup,from,subject,body,word_counts
1940,sci.med,balick@nynexst.com (Daphne Balick),Re: Altitude adjustment,In article <4159@mdavcr.mda.ca vida@mdavcr.mda...,"{'in': 7, 'article': 1, 'mdavcr': 2, 'mda': 2,..."
4532,rec.sport.hockey,Martin D. Hill <32GFKKH@CMUVM.CSV.CMICH.EDU>,Re: NHL team in Milwaukee,"Well put, Jason. I am not from Wisconsin, but...","{'well': 2, 'put': 1, 'jason': 1, 'i': 6, 'am'..."
3774,rec.motorcycles,edw@boi.hp.com (Edward Walsh),"Guzzi 850-T tires, plus fairing question",I have a question about tires for my Guzzi. It...,"{'i': 17, 'have': 5, 'a': 15, 'question': 2, '..."
4154,rec.motorcycles,rwf2@ns1.cc.lehigh.edu (ROBERT WILLIAM FUSI),Re: Recommendation for a front tire.,"In article <C5HrxJ.ACE@unix.amherst.edu, nrmen...","{'in': 4, 'article': 2, 'c': 2, 'hrxj': 1, 'ac..."
5204,rec.sport.baseball,klassen@sol.UVic.CA (Melvin Klassen),Re: Is Western insane??,In article <93108.1841463321628@QUCDN.QueensU....,"{'in': 1, 'article': 1, 'qucdn': 2, 'queensu':..."
...,...,...,...,...,...
2491,rec.autos,Tom Wetzel <twetzel@ucs.indiana.edu>,Re: Old Corvettes / Low insurance?,In article <rjnC5sxMG.JqH@netcom.com Richard N...,"{'in': 4, 'article': 1, 'rjnc': 1, 'sxmg': 1, ..."
2859,alt.atheism,keith@cco.caltech.edu (Keith Allan Schneider),"Re: ""Cruel"" (was Re: <Political Atheists?)",kmr4@po.CWRU.edu (Keith M. Ryan) writes:They s...,"{'kmr': 1, 'po': 1, 'cwru': 1, 'edu': 1, 'keit..."
4595,alt.atheism,kmr4@po.CWRU.edu (Keith M. Ryan),Re: Nicknames,In article <16BB6B6FE.I3150101@dbstu1.rz.tu-bs...,"{'in': 1, 'article': 1, 'bb': 1, 'b': 1, 'fe':..."
4346,alt.atheism,"dps@nasa.kodak.com (Dan Schaertel,,,)",Re: Christian Morality is,"In article 11853@vice.ICO.TEK.COM, bobbe@vice....","{'in': 1, 'article': 1, 'vice': 2, 'ico': 2, '..."


In [15]:
train_df.loc[90, "body"]

'zowie@daedalus.stanford.edu (Craig "Powderkeg" DeForest) writes:If you\'re planning on making long drives, the 20W50 is probably fine(esp. in the summer) in your 10W40 car.  But if you\'re making short drives,stick to the 10W40.Several years ago GM was having trouble with the rings sticking on the5.7 diesel.  They traced a cause to the use of 10W-40 oil.  They wouldnot honor warranty work if 10W-40 was used (if my memory serves me).5-30, 10-30 or 20 50 was OK\'d though.Mark B.'

In [16]:
counters = {}
for label in train_df["newsgroup"].unique():
    counters[label] = Counter()

for i in train_df.index:
    counters[train_df.loc[i, "newsgroup"]] += train_df.loc[i, "word_counts"]

In [17]:
word_counts = pd.DataFrame(counters)
word_counts

,sci.med,rec.sport.hockey,rec.motorcycles,rec.sport.baseball,sci.electronics,rec.autos,talk.religion.misc,alt.atheism
in,2351.0,1976.0,1357.0,1774.0,1259.0,1556.0,1712.0,2131.0
article,357.0,256.0,405.0,348.0,218.0,337.0,267.0,388.0
mdavcr,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mda,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ca,54.0,258.0,185.0,76.0,107.0,65.0,51.0,28.0
...,...,...,...,...,...,...,...,...
eagel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
whichcase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
hypnotized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
mousetrap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [18]:
with open("stopwords.txt") as foo:
    stops = foo.read().split(",")
    
word_counts = word_counts.drop(stops, errors="ignore").fillna(0)
word_counts.sample(10)

,sci.med,rec.sport.hockey,rec.motorcycles,rec.sport.baseball,sci.electronics,rec.autos,talk.religion.misc,alt.atheism
ra,2.0,7.0,4.0,9.0,1.0,1.0,50.0,3.0
hardware,0.0,0.0,2.0,0.0,30.0,2.0,0.0,4.0
hartje,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
o'dwyer,0.0,0.0,0.0,0.0,0.0,0.0,19.0,34.0
semisny,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
julie,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
thehorns,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
winegar,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
highlighted,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
compassionate,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0


In [19]:
## improve accuracy by adding 1 to all using laplace smoothing
words_counts = word_counts + 1
words_counts

,sci.med,rec.sport.hockey,rec.motorcycles,rec.sport.baseball,sci.electronics,rec.autos,talk.religion.misc,alt.atheism
article,358.0,257.0,406.0,349.0,219.0,338.0,268.0,389.0
mdavcr,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mda,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ca,55.0,259.0,186.0,77.0,108.0,66.0,52.0,29.0
vida,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
eagel,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
whichcase,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
hypnotized,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
mousetrap,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0


In [20]:
prob_autos = len(groups.get_group("rec.autos"))/len(train_df)
prob_scimed = len(groups.get_group("sci.med")) /len(train_df)
prob_atheism = len(groups.get_group("alt.atheism")) /len(train_df)
prob_baseball = len(groups.get_group("rec.sport.baseball")) /len(train_df)
prob_religion = len(groups.get_group("talk.religion.misc")) /len(train_df)
prob_hokey = len(groups.get_group("rec.sport.hockey")) /len(train_df)
prob_electronics = len(groups.get_group("sci.electronics")) /len(train_df)
prob_motorcycles = len(groups.get_group("rec.motorcycles")) /len(train_df)

In [21]:
def word_prob(w, newsgrp):
    return word_counts.loc[w, newsgrp]/word_counts[newsgrp].sum()

In [51]:
def predict_autos(text):
    words = re.findall("[a-z']+", text)
    prob_text_autos = sum([np.log10(word_prob(w, "rec.autos")) for w in words if w in word_counts.index])
    prob_newsgr_text = prob_text_autos + np.log10(prob_autos)
    return prob_newsgr_text

In [23]:
def predict_scimed(text):
    words = re.findall("[a-z']+", text.lower())
    prob_text_scimed = prod([word_prob(w, "sci.med") for w in words if w in word_counts.index])
    prob_newsgr_text = prob_text_scimed * prob_scimed
    return prob_newsgr_text

In [24]:
def predict_atheism(text):
    words = re.findall("[a-z']+", text.lower())
    prob_text_atheism = prod([word_prob(w, "alt.atheism") for w in words if w in word_counts.index])
    prob_newsgr_text = prob_text_atheism * prob_atheism
    return prob_newsgr_text

In [25]:
def predict_baseball(text):
    words =  re.findall("[a-z']+", text.lower())
    prob_text_baseball = prod([word_prob(w, "rec.sport.baseball") for w in words if w in word_counts.index])
    prob_newsgr_text = prob_text_baseball * prob_baseball
    return prob_newsgr_text

In [26]:
def predict_religion (text):
    words =  re.findall("[a-z']+", text.lower())
    prob_text_religion = prod([word_prob(w, "talk.religion.misc") for w in words if w in word_counts.index])
    prob_newsgr_text = prob_text_religion * prob_religion
    return prob_newsgr_text

In [27]:
def predict_hokey(text):
    words =  re.findall("[a-z']+", text.lower())
    prob_text_hokey = prod([word_prob(w, "rec.sport.hockey") for w in words if w in word_counts.index])
    prob_newsgr_text = prob_text_hokey * prob_hokey
    return prob_newsgr_text

In [28]:
def predict_electronics(text):
    words = re.findall("[a-z']+", text.lower())
    prob_text_electronics = prod([word_prob(w, "sci.electronics") for w in words if w in word_counts.index])
    prob_newsgr_text =  prob_text_electronics * prob_electronics
    return prob_newsgr_text

In [29]:
def predict_motorcycles(text):
    words = re.findall("[a-z']+", text.lower())
    prob_text_motorcycles = prod([word_prob(w, "rec.motorcycles") for w in words if w in word_counts.index])
    prob_newsgr_text = prob_text_motorcycles * prob_motorcycles
    return prob_newsgr_text

In [30]:
test_df = df.drop(train_df.index)
test_df

,newsgroup,from,subject,body
0,rec.autos,gwm@spl1.spl.loral.com (Gary W. Mahan),Re: Are BMW's worth the price?,sure sounds like they got a ringer. the 325is...
2,alt.atheism,"""Robert Knowles"" <p00261@psilink.com>",Re: Islamic marriage?,"DATE: Tue, 6 Apr 1993 00:11:49 GMTFROM: F...."
3,rec.sport.baseball,sepinwal@mail.sas.upenn.edu (Alan Sepinwall),Re: WFAN,In article <1993Apr16.174843.28111@cabell.vcu....
5,rec.sport.hockey,umturne4@ccu.umanitoba.ca (Daryl Turner),"Re: Jets/Canucks - Jets hold on, win 5-4",In article <C6067p.Lsp@news.cso.uiuc.edu eprit...
7,rec.sport.baseball,rickc@krill.corp.sgi.com (Richard Casares),Re: Jim Lefebvre is an idiot.,"In article <1993Apr5.190141.17623@bsu-ucs, 00b..."
...,...,...,...,...
7366,rec.sport.baseball,<RVESTERM@vma.cc.nd.edu>,Re: Jack Morris,In article <1993Apr20.004746.13007@ramsey.cs.l...
7367,sci.med,alan.barclay@almac.co.uk (Alan Barclay),Re: Need info on Circumci,TO: menon@boulder.Colorado.EDU (Ravi or Deanth...
7369,sci.med,jchen@wind.bellcore.com (Jason Chen),Re: Is MSG sensitivity superstition?,"In article <1qmlgaINNjab@hp-col.col.hp.com, ca..."
7372,sci.electronics,randy@ve6bc.ampr.ab.ca (Randy J. Pointkoski),Needed 24 volt 4 circuit Flasher,I am looking for a source for a 4 circuit Sequ...


In [52]:
test_df["prob_autos_t"] = predict_autos(str([test_df["body"]]))
test_df["prob_scimed_t"] = predict_scimed(str(test_df["body"]))
test_df["prob_atheism_t"] = predict_atheism(str(test_df["body"]))
test_df["prob_religion_t"] = predict_religion(str(test_df["body"]))
test_df["prob_baseball_t"] = predict_baseball(str(test_df["body"]))
test_df["prob_hokey_t"] = predict_hokey(str(test_df["body"]))
test_df["prob_motorcycles_t"] = predict_electronics(str(test_df["body"]))
test_df["prob_electronics_t"] = predict_motorcycles(str(test_df["body"]))

<ipython-input-51-e8868aebd48e>:3: RuntimeWarning: divide by zero encountered in log10
  prob_text_autos = sum([np.log10(word_prob(w, "rec.autos")) for w in words if w in word_counts.index])


In [42]:
test_df.sample(20)

,newsgroup,from,subject,body,prob_autos,prob_autos_t,prob_scimed_t,prob_atheism_t,prob_religion_t,prob_baseball_t,prob_hokey_t,prob_motorcycles_t,prob_electronics_t
3912,rec.autos,colombo@bronco.fnal.gov (Rick 'Open VMS 4ever'...,Re: Do trains/busses have radar?,"In article <C5FqFy.Fpq@usenet.ucs.indiana.edu,...",0.0,-inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5494,alt.atheism,karr@cs.cornell.edu (David Karr),Re: Fortune-guzzler barred from bars!,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,0.0,-inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
350,sci.med,geb@cs.pitt.edu (Gordon Banks),Re: health care reform,In article <LMC006@wrc.wrgrace.com custer@wrc....,0.0,-inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1543,alt.atheism,mccullou@snake2.cs.wisc.edu (Mark McCullough),Re: Amusing atheists and anarchists,In article <timmbake.735294667@mcl timmbake@mc...,0.0,-inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4264,sci.med,ls8139@albnyvms.bitnet (larry silverberg),Re: Good Grief! (was Re: Candida Albicans: wh...,"In article <noringC5snsx.KMo@netcom.com, norin...",0.0,-inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,rec.sport.baseball,lbr@holos0.uucp (Len Reed),Re: Rule Book Trivia 2,"After reading my local paper today, I found ou...",0.0,-inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5929,rec.sport.hockey,Anna Matyas <am2x+@andrew.cmu.edu>,Re: Pens fans reactions,Susan Dart writes:Amongst Penguins' fans there...,0.0,-inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6440,talk.religion.misc,mike@inti.lbl.gov (Michael Helm),Re: Religion and history; The real discuss,Matthew Huntbach writes:sm[?]a real Christian ...,0.0,-inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4614,rec.sport.baseball,maynard@ramsey.cs.laurentian.ca (Roger Maynard),Re: Jack Morris,In <48178@sdcc12.ucsd.edu demers@cs.ucsd.edu (...,0.0,-inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1608,talk.religion.misc,pmoloney@maths.tcd.ie (Paul Moloney),Re: Biblical Backing of Koresh's 3-02 Tape (Ci...,"cotera@woods.ulowell.edu writes:Once again, wh...",0.0,-inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
word_counts["sci.med"].sum()

76866.0

In [45]:
word_counts["sci.med"].value_counts()

0.0      42799
1.0       8871
2.0       2470
3.0       1108
4.0        738
         ...  
357.0        1
86.0         1
172.0        1
244.0        1
150.0        1
Name: sci.med, Length: 142, dtype: int64

In [48]:
word_counts.loc["article", "sci.med"]/word_counts["sci.med"].sum()

0.004644446179064866

In [50]:
word_counts.loc["article", "sci.med"]/word_counts["sci.med"].value_counts()

0.0        0.008341
1.0        0.040243
2.0        0.144534
3.0        0.322202
4.0        0.483740
            ...    
357.0    357.000000
86.0     357.000000
172.0    357.000000
244.0    357.000000
150.0    357.000000
Name: sci.med, Length: 142, dtype: float64